In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.signal import find_peaks
import MetaTrader5 as mt5
import pytz
import time

ModuleNotFoundError: No module named 'scipy'

In [2]:
# Initialize MT5 connection
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

### Defining functions

In [3]:
def rename_column_price_n(Pay):
    Pa = p1_range_localminimas[p1_range_localminimas['low'] == Pay].drop(['open','high','close'], axis=1)
    Pa = Pa.drop_duplicates(subset=['low'], keep='last')
    Pa = Pa.rename(columns={"low":"price"})
    return Pa

def rename_column_price_x(Pay, condition):
    Pa = p1_range_localmaximas[(p1_range_localmaximas['high'] == Pay) & condition].drop(['open','low','close'], axis=1)
    Pa = Pa.drop_duplicates(subset=['high'], keep='last')
    Pa = Pa.rename(columns={"high":"price"})
    return Pa

def compute_index_Pa_n(Pay):
    Pay_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay].index[-1]
    return Pay_index

def create_tuples_p1_p3(df):
    p1_p2_p3 = []
    max_index = df.index.max()
    no_of_comb = ((max_index + 1) // 2) -1  # +1 to include the last element for odd lengths
    for i in range(no_of_comb):
        p1_p2_p3.append((max_index, max_index - 2*(i+1) + 1, max_index - 2*(i+1)))
    return p1_p2_p3 

def fetch_latest_data(OHLC_df):
    timezone = pytz.timezone('UTC')
    start_time = datetime.now(timezone) + timedelta(hours=3) - timedelta(minutes=5)
    end_time = datetime.now(timezone) + timedelta(hours=3) - timedelta(minutes=1)
    rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
    new_data = pd.DataFrame(rates)
    new_data['time'] = pd.to_datetime(new_data['time'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    new_data.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    new_data = new_data.reindex(labels=["time","open", "high", "low", "close"], axis = 1)
    new_data = new_data.rename(columns={"time":"datetime"})
    new_data["datetime"] = new_data["datetime"].astype('<M8[ns]')
    updated_ohlc_data = pd.concat([OHLC_df,new_data],ignore_index=True)
    updated_ohlc_data = updated_ohlc_data.drop_duplicates(ignore_index= True)
    updated_ohlc_data['datetime'] = updated_ohlc_data['datetime'].astype('<M8[ns]')
    return updated_ohlc_data

### Downloading the OHLC data from the last 7 days to 2 minutes ago

In [4]:
symbol = 'EURUSD'.upper()
timeframe = mt5.TIMEFRAME_M1

#Get the time ranges in utc
timezone = pytz.timezone('UTC')
start_time = datetime.now(timezone) - timedelta(days=7) + timedelta(hours=3)
end_time = datetime.now(timezone) - timedelta(minutes=2) + timedelta(hours=3)

rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)

# Create a DataFrame
rates_df = pd.DataFrame(rates)
rates_df['time'] = pd.to_datetime(rates_df['time'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
rates_df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
rates_df = rates_df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)
rates_df['time'] = rates_df['time'].astype('<M8[ns]')
rates_df = rates_df.rename(columns={'time':'datetime'})

In [1]:
rates_df = fetch_latest_data(rates_df)
rates_df.tail()

NameError: name 'fetch_latest_data' is not defined

### CONTINUE THIS TOMORROW
#### IN OBTAINING THE HIGHEST POINT, THERE WILL BE A LOT OF DUPLICATES SINCE THE DATA IS FROM THE LAST 7 DAYS. 
#### FIND THE MAX POINT FROM THE LAST 1HR 21 MIN
##### IF THE MAX POINT IS NOT MOST RECENT, WAIT FOR THE OPPORTUNITY

In [6]:
#obtaining the OHLC data
start_dt="2024-09-08 9:00:00"
end_dt="2024-09-08 14:00:00"

rates_df = OHLC_df[(OHLC_df['datetime'] >= start_dt) & (OHLC_df['datetime'] <= end_dt)]

#OBTAINING THE HIGHEST POINT DURING THE NY SESSION
p3y = rates_df['high'].max()

#OBTAINING THE INDEX OF THE HIGHEST POINT
p3i = (rates_df[rates_df['high']== p3y])

#obtaining the OHLC data of P3
p3 = p3i.iloc[[-1]]

#converting the datatype of the datetime column
rates_df.loc[:,'datetime'] = pd.to_datetime(rates_df["datetime"])

#obtaining the data within the time limit 4 to 45 mins before P3
p1x_range_1 = p3["datetime"].iloc[0] - timedelta(minutes = 45)
p1x_range_2 = p3["datetime"].iloc[0] - timedelta(minutes = 0)
mask1= rates_df["datetime"].ge(p1x_range_1) & rates_df["datetime"].le(p1x_range_2)
p1_range = rates_df[mask1].reset_index(drop=True)

In [7]:
# Find the index of local maximas
local_maximas_i,_ = find_peaks(p1_range['high'].values)
local_maximas_i = np.array(local_maximas_i)

# Find the index of the local minima by inverting the data
local_minimas_i,_ = find_peaks(-p1_range['low'].values)
local_minimas_i = np.array(local_minimas_i)

#Obtaining the dataframe of the local minimas and local maximas
p1_range_localminimas = p1_range[p1_range.index.isin(local_minimas_i)].reset_index(drop=True)
p1_range_localmaximas = p1_range[p1_range.index.isin(local_maximas_i)].reset_index(drop=True)

In [8]:
#Finding the value of Pay1
Pay1 = p1_range_localminimas["low"].min()

#Finding the index of the first local minima
Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index.min()


#Dropping the minimas before the first local minimas
mask = p1_range_localminimas.index >= Pay1_index
p1_range_localminimas = p1_range_localminimas[mask].reset_index(drop=True)
p1_range_localminimas
#Finding the index of the first local minima
Pay1_index = compute_index_Pa_n(Pay1)

In [9]:
#to find the y value of the possible 2nd local minima:
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = compute_index_Pa_n(Pay3)
Pay3_index_range = Pay3_index

In [10]:
while True:
    try:
        #finding the index of the 1st local maxima
        condition1 = p1_range_localmaximas["datetime"].ge(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].le(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2
        
        #to find the y value of the 1st local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        if len(Pay2) == 0:
            #proceed with the next pay3 index and rerun the while loop
            Pay3_index = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index:,3].min()
            print(Pay3)
            continue
        else:
            Pay2=Pay2.max()
            break
    except IndexError as err1:
        break

1.0966


In [11]:
#Making a new dataframe of the higher highs and higher lows approaching P3 in chronological order
HH_HL_df = pd.DataFrame(columns=['datetime','price'])
HH_HL_df = pd.DataFrame({'datetime': pd.Series(dtype='datetime64[ns]'), 'price': pd.Series(dtype='float')})

#Grabbing the data attached to Pay*
HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay1)], ignore_index=True)

Pa2 = p1_range_localmaximas[p1_range_localmaximas['high'] == Pay2].drop(['open','low','close'], axis=1)
Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
Pa2 = Pa2.rename(columns={"high":"price"})
HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)

HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay3)], ignore_index=True)

In [12]:
#Resetting the values to avoid issues
Pay2 = Pay1_index = Pay3_index = 0

#Assigning new values to find new sets of Pa2, Pa3
Pay1 = Pay3
Pay1_index = compute_index_Pa_n(Pay1)

#Resetting the values to avoid issues
Pay3 = Pay3_index = 0

In [13]:
#Finding the initial value of Pay3
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = compute_index_Pa_n(Pay3)


#Finding the values of the higher lows and higher highs and appending it to the HH_HL_df
while True:
    Pay3_index_range = 0
    try:
        # Check if the current Pay3_index is beyond the end of p1_range_localminimas
        if Pay3_index > len(p1_range_localminimas):
            print(f"End of p1_range_localminimas reached.{Pay3_index}")
            break
            
        #finding the index of the next local maxima
        condition1 = p1_range_localmaximas["datetime"].ge(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].le(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2

        
        #to find the y value of the next local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        
        #previous HH index
        pHH_index = HH_HL_df[HH_HL_df['price'] == Pay1].index[0] - 1
        pHHy = HH_HL_df.iloc[pHH_index,1]

        #if Pay2 isnt existing for the current Pay3
        condition3 = len(Pay2) == 0
        
        #if the current Pay2 is less than the previous HH
        try:
            condition4 = p1_range_localmaximas[condition == True]['high'].values.max() < pHHy
        except ValueError as err2:
            Pay3_index = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
        if condition3 or condition4:
            #proceed with the next pay3 index and rerun the while loop
            Pay2 = 0
            Pay3_index_range = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index_range:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
        else:
            Pay2=Pay2.max()
            #Appending to the HH_HL_df 
            condition5 = (p1_range_localmaximas['high'] == Pay2) & condition
            HH_HL_df = pd.concat([HH_HL_df, rename_column_price_x(Pay2, condition5)], ignore_index=True)
            HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay3)], ignore_index=True)
            
            #Assigning new values to find new sets of Pa2, Pa3
            Pay1 = Pay3
            Pay1_index = compute_index_Pa_n(Pay1)
            
            #Finding the initial value of Pay3
            Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
    except IndexError as err1:
        break

In [4]:
#Append p3 to the dataframe:
p3 = p3.drop(['open','low','close'], axis=1)
p3 = p3.rename(columns={"high":"price"})
HH_HL_df = pd.concat([HH_HL_df, p3], ignore_index=True)

# Create a tuple combining the list of possible p1-p2-p3 (initial values)
p1_p2_p3i = create_tuples_p1_p3(HH_HL_df)
p1_p2_p3i

NameError: name 'p3' is not defined

In [2]:
# finding the right p1-p2-p3 in terms of ratios

p1_p2_p3_index = []
for i in p1_p2_p3i:
    p3i=i[0]
    p2i=i[1]
    p1i=i[2]
    p3y_p2y = HH_HL_df.iloc[p3i,1] - HH_HL_df.iloc[p2i,1]
    p2y_p1y = HH_HL_df.iloc[p2i,1] - HH_HL_df.iloc[p1i,1]
    p3x_p2x = HH_HL_df.iloc[p3i,0] - HH_HL_df.iloc[p2i,0]
    p2x_p1x = HH_HL_df.iloc[p2i,0] - HH_HL_df.iloc[p1i,0]

    # conditions for finding the right p1-p2-p3
    condition1 = 1.027 < abs(p3y_p2y/p2y_p1y) < 2.337
    condition2 = 0.333 < p3x_p2x/p2x_p1x < 5
    condition3 = p2x_p1x > pd.Timedelta(minutes=1)
    condition4 = pd.Timedelta(minutes=2) < p2x_p1x < pd.Timedelta(minutes=24)
    condition5 = pd.Timedelta(minutes=2) < p2x_p1x < pd.Timedelta(minutes=21)
    condition = condition1 & condition2 & condition3 & condition4 & condition5
    
    if condition:
        p1_p2_p3_index.append(i)
        print(p1_p2_p3_index)

NameError: name 'p1_p2_p3i' is not defined

# Values for Testing Purposes

In [16]:
#previous HH index
HH_HL_df.head(10)

,datetime,price
0,2024-08-05 15:07:00,1.09647
1,2024-08-05 15:10:00,1.09694
2,2024-08-05 15:21:00,1.09660
3,2024-08-05 15:25:00,1.09747
4,2024-08-05 15:32:00,1.09671
5,2024-08-05 15:41:00,1.09844
6,2024-08-05 15:42:00,1.09801
7,2024-08-05 15:49:00,1.10084
